In [1]:
import json
import logging
import os
import numpy as np
import tensorflow as tf
from scipy.stats import norm
from config import get_config
from equation import get_equation
from solver import FeedForwardModel

In [2]:
s0=100
k=100
r=0.02
sigma=0.20
T=1

d1=(np.log(s0/k)+(r+sigma*sigma/2)*T)/(sigma*np.sqrt(T))
d2=(np.log(s0/k)+(r-sigma*sigma/2)*T)/(sigma*np.sqrt(T))

print(norm.cdf(d1)*s0-norm.cdf(d2)*k*np.exp(-r*T))

8.916037278572539


In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

tf.app.flags.DEFINE_string('problem_name', 'EuropeanCall',
                           """The name of partial differential equation.""")
tf.app.flags.DEFINE_integer('num_run', 1,
                            """The number of experiments to repeatedly run for the same problem.""")
tf.app.flags.DEFINE_string('log_dir', './logs',
                           """Directory where to write event logs and output array.""")

In [4]:
def main():
    problem_name = FLAGS.problem_name
    config = get_config(problem_name)
    bsde = get_equation(problem_name, config.dim, config.total_time, config.num_time_interval)

    if not os.path.exists(FLAGS.log_dir):
        os.mkdir(FLAGS.log_dir)
    path_prefix = os.path.join(FLAGS.log_dir, problem_name)
    with open('{}_config.json'.format(path_prefix), 'w') as outfile:
        json.dump(dict((name, getattr(config, name))
                       for name in dir(config) if not name.startswith('__')),
                  outfile, indent=2)
    logging.basicConfig(level=logging.INFO,
                        format='%(levelname)-6s %(message)s')

    for idx_run in range(1, FLAGS.num_run+1):
        tf.reset_default_graph()
        with tf.Session() as sess:
            logging.info('Begin to solve %s with run %d' % (problem_name, idx_run))
            model = FeedForwardModel(config, bsde, sess)
            if bsde.y_init:
                logging.info('Y0_true: %.4e' % bsde.y_init)
            model.build()
            training_history = model.train()
            if bsde.y_init:
                logging.info('relative error of Y0: %s',
                             '{:.2%}'.format(
                                 abs(bsde.y_init - training_history[-1, 2])/bsde.y_init))
            # save training history
            np.savetxt('{}_training_history_{}.csv'.format(path_prefix, idx_run),
                       training_history,
                       fmt=['%d', '%.5e', '%.5e', '%d'],
                       delimiter=",",
                       header="step,loss_function,target_value,elapsed_time",
                       comments='')

In [5]:
main()

INFO   Begin to solve EuropeanCall with run 1


WARNING From C:\Users\sy8w\OneDrive\Documents\Projects and Presentation\BSDE\solver.py:54: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



WARNING From C:\Users\sy8w\OneDrive\Documents\Projects and Presentation\BSDE\solver.py:57: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



WARNING From C:\Users\sy8w\OneDrive\Documents\Projects and Presentation\BSDE\solver.py:67: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



Instructions for updating:
keep_dims is deprecated, use keepdims instead


...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


WARNING From C:\Users\sy8w\OneDrive\Documents\Projects and Presentation\BSDE\solver.py:80: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


WARNING From C:\Users\sy8w\OneDrive\Documents\Projects and Presentation\BSDE\solver.py:85: The name tf.train.piecewise_constant is deprecated. Please use tf.compat.v1.train.piecewise_constant instead.



WARNING From C:\Users\sy8w\OneDrive\Documents\Projects and Presentation\BSDE\solver.py:88: The name tf.trainable_variables is deprecated. Please use tf.compat.v1.trainable_variables instead.



WARNING From C:\Users\sy8w\OneDrive\Documents\Projects and Presentation\BSDE\solver.py:90: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

INFO   step:     0,    loss: 1.7619e+02,   Y0: 8.9696e+00,  elapsed time  39
INFO   step:   100,    loss: 1.8109e+02,   Y0: 8.8918e+00,  elapsed time  60
INFO   step:   200,    loss: 1.6774e+02,   Y0: 8.8221e+00,  elapsed time  63
INFO   step:   300,    loss: 1.5641e+02,   Y0: 8.7740e+00,  elapsed time  66
INFO   step:   400,    loss: 1.4486e+02,   Y0: 8.8250e+00,  elapsed time  68
INFO   step:   500,    loss: 1.3545e+02,   Y0: 8.8615e+00,  elapsed time  71
INFO   step:   600,    loss: 1.2198e+02,   Y0: 8.9314e+00,  elapsed time  74
INFO   step:   700,    loss: 1.1238e+02,   Y0: 8.9787e+00,  elapsed time  76
INFO   step:   800,    loss: 1.0246e+02,   Y0: 8.9425e+00,  elapsed time  79
INFO   step:   900,    loss: 9.3629e+01,   Y0: 8.8912e+00,  elapsed time  82
INFO   step:  1000,    loss: 8.5822e+0

In [8]:
s0=100
k=100
r=0.02
sigma=0.20
T=1

d1=(np.log(s0/k)+(r+sigma*sigma/2)*T)/(sigma*np.sqrt(T))
d2=(np.log(s0/k)+(r-sigma*sigma/2)*T)/(sigma*np.sqrt(T))

print(norm.cdf(d1)*s0-norm.cdf(d2)*k*np.exp(-r*T))

8.916037278572539
